# Генератор одномерных данных

В данном семинаре мы рассмотрим простой пример применения генеративно-состязательных сетей.

<img src="https://raw.githubusercontent.com/torch/torch.github.io/master/blog/_posts/images/model.png" width=320px height=240px>

Простым примером являются генеративно-состязательные сети, которые обучатся преобразованию одномерного равномерного распределения одномерное нормальное распределение. 

Данный пример содержит множество полезных визуализаций, которые помогут изучить работу двух сетей и исправить распространённые ошибки без необходимости ждать окончания многочасовых расчётов.

In [1]:
import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

def sample_noise(batch_size):
    """ Uniform noise of shape [batch_size, 1] in range [0, 1]"""
    return Variable(torch.rand(batch_size, 1))

def sample_real_data(batch_size):
    """ Normal noise of shape [batch_size, 1], mu=5, std=1.5 """
    return Variable(torch.randn(batch_size, 1) * 1.5 + 5)

In [ ]:
# Generator converts 1d noise into 1d data
gen = nn.Sequential(nn.Linear(1, 16), nn.ELU(), nn.Linear(16, 1))
gen_opt = torch.optim.SGD(gen.parameters(), lr=1e-3)

# Discriminator converts 1d data into two logits (0th for real, 1st for fake). 
# It is deliberately made stronger than generator to make sure disc is slightly "ahead in the game".
disc = nn.Sequential(nn.Linear(1, 64), nn.ELU(), nn.Linear(64, 2), nn.Sigmoid())
disc_opt = torch.optim.SGD(disc.parameters(), lr=1e-2)

In [ ]:
# we define 0-th output of discriminator as "is_fake" output and 1-st as "is_real"
IS_FAKE, IS_REAL = 0, 1

def train_disc(batch_size):
    """ trains discriminator for one step """
    
    # compute p(real | x)
    real_data = sample_real_data(batch_size)
    p_real_is_real = disc(real_data)
    
    # compute p(fake | G(z)). We detach to avoid computing gradinents through G(z)
    noise = <sample noise>
    gen_data = <generate data given noise>
    p_gen_is_fake = <compute logp for 0th>
    
    disc_loss = <compute loss>
    
    # sgd step. We zero_grad first to clear any gradients left from generator training
    disc_opt.zero_grad()
    disc_loss.backward()
    disc_opt.step()
    return disc_loss.data.numpy()[0]

In [ ]:
def train_gen(batch_size):
    """ trains generator for one step """
        
    # compute p(fake | G(z)).
    noise = <sample noise>
    gen_data = <generate data given noise>
    
    p_gen_is_real = <compute logp gen is REAL>
    
    gen_loss = <generator loss>
    
    gen_opt.zero_grad()
    gen_loss.backward()
    gen_opt.step()
    return gen_loss.data.numpy()[0]

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
%matplotlib inline

for i in range(100000):

    for _ in range(5):
        train_disc(128)
    
    train_gen(128)
    
    if i % 250 == 0:
        clear_output(True)
        plt.figure(figsize=[14, 6])
        plt.subplot(1, 2, 1)
        plt.title("Data distributions")
        plt.hist(gen(sample_noise(1000)).data.numpy()[:, 0], range=[0, 10], alpha=0.5, normed=True, label='gen')
        plt.hist(sample_real_data(1000).data.numpy()[:,0], range=[0, 10], alpha=0.5, normed=True, label='real')
        
        x = np.linspace(0,10, dtype='float32')
        disc_preal = disc(Variable(torch.from_numpy(x[:, None])))
        plt.plot(x, disc_preal.data.numpy(), label='disc P(real)')
        plt.legend()
        

        plt.subplot(1, 2, 2)
        plt.title("Discriminator readout on real vs gen")
        plt.hist(disc(gen(sample_noise(100))).data.numpy(),
                 range=[0, 1], alpha=0.5, label='D(G(z))')
        plt.hist(disc(sample_real_data(100)).data.numpy(),
                 range=[0, 1], alpha=0.5, label='D(x)')
        plt.legend()
        plt.show()

__Что ожидать:__
* __Слева:__ Поначалу два распределения будут выглядеть по-разному, но потом распределение генератора должно совпасть с реальными данными _почти_ везде. Кривая представляет собой решение дискриминатора для всех возможных значений x. Постепенно оно должно приближаться к 0.5 в областях, содержащих много реальных значений.
* __Справа:__ Этот график показывает, как часто дискриминатор назначает определённое значение вероятности истинным и искуственным образцам (они отличаются цветом). Для первых итераций допустимы колебания. После этого, когда генератор достаточно обучится, большая часть значений приблизится к 0.5.
 * Если вместо этого значения сходятся к двум дельта-функциям около 0 (искусственные образцы) и 1 (истинные образцы), то дискриминатор одержал верх над генератором. _Проверьте функцию потерь генератора_. Другим решением является уменьшение скорости обучения дискриминатора. Это также может произойти при замене среднего для батча на сумму или схожую функцию.
 * Если значения сходятся к 0.5 и остаются там на протяжении нескольких итераций, но генератор не научился генерировать правдоподобные образцы, то генератор одержал верх над дискриминатором. _Дополнительно проверьте функцию потерь дискриминатора_.